In [3]:
%pdb off
%matplotlib inline

import os, os.path as op
import socket, glob
from pathlib import Path
import xarray as xr
from datetime import datetime
# from VLM import *

from IPython.display import Image, display
from mintpy.cli import view, tsview
from mintpy.utils import writefile, readfile, utils as ut
from dataclasses import dataclass


Automatic pdb calling has been turned OFF


In [12]:
print (os.getenv('CONDA_PROMPT_MODIFIER'), datetime.now())
print ('Track:', track)

(ARIA)  2023-09-25 19:25:04.101337
Track: 135


In [5]:
# track to GPS station, its lalo
dct_gps  = {33: ['NYBK', '40.703432 -73.978965'], 
            135: ['CTWF', '41.3358099728 -72.1155494566'], #['URIL', '41.4889420 -71.5277192'], 
            62: ['MAMI', '42.2720614 -71.0486846'], 
            164: ['MERO', '44.1137947 -69.1186410'], 
            91: ['HAL3', '44.6572781589 -63.6650616709']
}


# track to region name
dct_reg  = {33: 'NYC', 135: 'RI', 62: 'Mass', 164: 'Maine', 91: 'NS'}

In [24]:
@dataclass
class ECExp:
    track: int
    mp_exp0: str

    def __post_init__(self):
        reg      = dct_reg[track]
        ref_sta  = dct_gps[track][0]
        mp_exp   = f'{ref_sta}_{self.mp_exp0}'
        trackS   = str(track).zfill(3)
        self.path_wd    = Path(os.getenv('dataroot')) / 'VLM' / 'Sentinel1' / 'EastCoast' / f'track_{trackS}'
        self.path_wd_mp = self.path_wd / mp_exp
        self.path_wd_mp.mkdir(exist_ok=True)

        self.path_spacoh = self.path_wd_mp / 'avgSpatialCoh.h5'
        self.path_tcoh   = self.path_wd_mp / 'temporalCoherence.h5'

        
        print (f'\n{reg} Experiment:', mp_exp)
        print ('MintPy Directory:', str(self.path_wd_mp))
        
        self.set_stack_paths()
        self.set_mask_paths()
        self.set_disp_paths()
        
    
    def set_stack_paths(self):
        self.path_ifgs  = self.path_wd_mp / 'inputs' / 'ifgramStack.h5'
        self.path_geom  = self.path_wd_mp / 'inputs' / 'geometryGeo.h5'

        
    def set_mask_paths(self):
        self.path_wmask  = self.path_wd_mp / 'waterMask.h5'
        self.path_mask   = self.path_wd_mp / 'maskWaterTempCoh.h5' # combine

        
        
    def set_disp_paths(self):
        if 'ERA5' in self.mp_exp0:
            self.path_ts      = self.path_wd_mp / 'timeseries_ERA5_demErr.h5'
        else:
            print ('Using only dem-error corrected timeseries')
            self.path_ts      = self.path_wd_mp / 'timeseries_demErr.h5'
        self.path_vlos    = self.path_wd_mp / 'velocity.h5'
        self.path_vlos_pm = self.path_wd_mp / 'velocity_ITRF14.h5'

        
track    = 135

track135    = ECExp(track, 'ERA5_PM_ex') # no unw corr
track135_br = ECExp(track, 'ERA5_PM_bridging_ex') # phase closure
track135_pc = ECExp(track, 'ERA5_PM_closure_ex') # bridging



RI Experiment: CTWF_ERA5_PM_ex
MintPy Directory: /u/leffe-data2/buzzanga/data/VLM/Sentinel1/EastCoast/track_135/CTWF_ERA5_PM_ex

RI Experiment: CTWF_ERA5_PM_bridging_ex
MintPy Directory: /u/leffe-data2/buzzanga/data/VLM/Sentinel1/EastCoast/track_135/CTWF_ERA5_PM_bridging_ex

RI Experiment: CTWF_ERA5_PM_closure_ex
MintPy Directory: /u/leffe-data2/buzzanga/data/VLM/Sentinel1/EastCoast/track_135/CTWF_ERA5_PM_closure_ex
